<a href="https://colab.research.google.com/github/sonalshreya/humor_in_AI/blob/main/Evaluating_Blip2_and_ViT_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import (
    VisionEncoderDecoderModel,
    BlipForConditionalGeneration,
    BlipProcessor,
    AutoFeatureExtractor,
    AutoTokenizer
)

In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=d3f809f3059c505f30fbd49cad4585b5a3d06a35cc459fad56ffe25bea2a1a1a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from PIL import Image
import numpy as np
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

In [ ]:
try:
    # Try downloading punkt_tab
    nltk.download('punkt_tab', quiet=True)
except:
    pass

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

In [ ]:
cartoon_dataset = load_dataset("jmhessel/newyorker_caption_contest", 'explanation', split='train')
cartoon_image = cartoon_dataset[2]['image']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/40.2k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/125M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/7.97M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/6.80M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2340 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/130 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/131 [00:00<?, ? examples/s]

In [ ]:
class ImageDescriptionEvaluator:
    def __init__(self, ground_truth):
        """
        Initialize evaluator with ground truth description

        Args:
            ground_truth (str): Manually annotated description of the image
        """
        self.ground_truth = ground_truth

        # Initialize evaluation metrics
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    def generate_description_vit(self, image):
        """
        Generate description using Vision Transformer model

        Args:
            image (PIL.Image): Input image

        Returns:
            str: Generated description
        """

        image = self.preprocess_image(image)
        # Load ViT model
        model_name = "nlpconnect/vit-gpt2-image-captioning"
        model = VisionEncoderDecoderModel.from_pretrained(model_name)
        feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Preprocess image
        inputs = feature_extractor(image, return_tensors="pt")

        # Generate description
        outputs = model.generate(**inputs, max_length=50)
        description = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return description

    def preprocess_image(self, image):
        """
        Ensure image is in correct format for models

        Args:
            image (PIL.Image or numpy.ndarray): Input image

        Returns:
            PIL.Image: Preprocessed image
        """
        # If image is a numpy array, convert to PIL Image
        if isinstance(image, np.ndarray):
            if image.ndim == 2:
                # Grayscale to RGB
                image = Image.fromarray(image).convert('RGB')
            elif image.ndim == 3:
                # Check if it's already in RGB format
                if image.shape[2] == 3:
                    image = Image.fromarray(image.astype('uint8'), 'RGB')
                else:
                    raise ValueError(f"Unexpected image shape: {image.shape}")

        # If it's not already a PIL Image, convert
        if not isinstance(image, Image.Image):
            image = Image.fromarray(image)

        # Ensure RGB
        if image.mode != 'RGB':
            image = image.convert('RGB')

        return image

    def generate_description_blip2(self, image):
        """
        Generate description using BLIP-2 model

        Args:
            image (PIL.Image): Input image

        Returns:
            str: Generated description
        """

        image = self.preprocess_image(image)
        # Load BLIP-2 model
        model_name = "Salesforce/blip-image-captioning-large"
        model = BlipForConditionalGeneration.from_pretrained(model_name)
        processor = BlipProcessor.from_pretrained(model_name)

        # Preprocess image
        inputs = processor(image, return_tensors="pt")
        #inputs = processor(images=image, text="Describe this image in detail:", return_tensors="pt")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=100,
                num_beams=4,  # Use beam search
                early_stopping=True
            )
        # Generate description
        #outputs = model.generate(**inputs, max_length=50)
        description = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)

        return description

    def evaluate_description(self, generated_description):
        """
        Evaluate generated description against ground truth

        Args:
            generated_description (str): Description to evaluate

        Returns:
            dict: Evaluation metrics
        """
        # Tokenize descriptions
        ground_truth_tokens = word_tokenize(self.ground_truth.lower())
        generated_tokens = word_tokenize(generated_description.lower())

        # Calculate ROUGE scores
        rouge_scores = self.rouge_scorer.score(self.ground_truth, generated_description)

        # Calculate BLEU score
        bleu_score = sentence_bleu([ground_truth_tokens], generated_tokens)

        # Calculate unique word coverage
        unique_gt_words = set(ground_truth_tokens)
        unique_gen_words = set(generated_tokens)
        word_coverage = len(unique_gen_words.intersection(unique_gt_words)) / len(unique_gt_words)

        return {
            'ROUGE-1': rouge_scores['rouge1'].fmeasure,
            'ROUGE-2': rouge_scores['rouge2'].fmeasure,
            'ROUGE-L': rouge_scores['rougeL'].fmeasure,
            'BLEU Score': bleu_score,
            'Word Coverage': word_coverage
        }

    def compare_models(self, image):
        """
        Compare ViT and BLIP-2 models for image description

        Args:
            image (PIL.Image): Input image

        Returns:
            dict: Comparative evaluation results
        """
        # Generate descriptions
        vit_description = self.generate_description_vit(image)
        blip2_description = self.generate_description_blip2(image)

        # Evaluate descriptions
        vit_metrics = self.evaluate_description(vit_description)
        blip2_metrics = self.evaluate_description(blip2_description)

        return {
            'ViT Description': vit_description,
            'BLIP-2 Description': blip2_description,
            'ViT Metrics': vit_metrics,
            'BLIP-2 Metrics': blip2_metrics
        }

In [ ]:
# Example usage
# Replace with your actual ground truth description
ground_truth = "There are two men in suits standing at the door of an office both looking at a desk with a medieval sword stuck into the middle of the desk. One man is talking."

# Load your cartoon image
# Assuming cartoon_image is your PIL Image object
evaluator = ImageDescriptionEvaluator(ground_truth)
comparison_results = evaluator.compare_models(cartoon_image)

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.46.2"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_rang

In [ ]:
# Print results
print("Comparison Results:")
print("\nViT Description:", comparison_results['ViT Description'])
print("\nBLIP-2 Description:", comparison_results['BLIP-2 Description'])


Comparison Results:

ViT Description: a painting of a painting of a man and a woman 

BLIP-2 Description: there is a man standing in front of a desk in an office


In [ ]:
print("\nViT Metrics:")
for metric, value in comparison_results['ViT Metrics'].items():
    print(f"{metric}: {value}")

print("\nBLIP-2 Metrics:")
for metric, value in comparison_results['BLIP-2 Metrics'].items():
    print(f"{metric}: {value}")


ViT Metrics:
ROUGE-1: 0.22727272727272727
ROUGE-2: 0.0
ROUGE-L: 0.18181818181818182
BLEU Score: 1.6879168804322106e-232
Word Coverage: 0.10714285714285714

BLIP-2 Metrics:
ROUGE-1: 0.47826086956521735
ROUGE-2: 0.09090909090909091
ROUGE-L: 0.21739130434782608
BLEU Score: 1.6828168187113343e-155
Word Coverage: 0.35714285714285715


Meaning of below scores
ViT Metrics:
ROUGE-1: 0.22727272727272727
ROUGE-2: 0.0
ROUGE-L: 0.18181818181818182
BLEU Score: 1.6879168804322106e-232
Word Coverage: 0.10714285714285714

BLIP-2 Metrics:
ROUGE-1: 0.37499999999999994
ROUGE-2: 0.13043478260869562
ROUGE-L: 0.25000000000000006
BLEU Score: 0.053113284



ROUGE Scores (Recall-Oriented Understudy for Gisting Evaluation):

ROUGE-1: Measures the overlap of individual words between generated and ground truth descriptions

ViT: 0.227 (22.7% word overlap)
BLIP-2: 0.375 (37.5% word overlap)


ROUGE-2: Measures the overlap of two-word phrases

ViT: 0.0 (no two-word phrase matches)
BLIP-2: 0.131 (13% two-word phrase matches)


ROUGE-L: Measures the longest common subsequence

ViT: 0.182 (18.2% longest common subsequence)
BLIP-2: 0.250 (25% longest common subsequence)




BLEU Score (Bilingual Evaluation Understudy):

Measures the precision of generated text compared to reference text
ViT: Extremely low (essentially zero)
BLIP-2: 0.053 (slightly better, but still low)


Word Coverage:

Percentage of unique words from ground truth that appear in the generated description
ViT: 0.107 (10.7% of unique words covered)
BLIP-2: 0.25 (25% of unique words covered)



Interpretation:

BLIP-2 performs significantly better across all metrics
ViT struggled to generate a description close to the ground truth
BLIP-2 captured more words and phrases from the original description
Both models have relatively low scores, suggesting the generated descriptions are quite different from the ground truth